In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf

In [2]:
df = pd.read_csv('match_dataframe.csv')
df.dtypes

Unnamed: 0          int64
0.assists           int64
0.baronKills        int64
0.basicPings      float64
0.bountyLevel       int64
                   ...   
5.summonerCode      int64
6.summonerCode      int64
7.summonerCode      int64
8.summonerCode      int64
9.summonerCode      int64
Length: 1382, dtype: object

In [3]:
df = df.astype(np.float32)
df.dtypes

Unnamed: 0        float32
0.assists         float32
0.baronKills      float32
0.basicPings      float32
0.bountyLevel     float32
                   ...   
5.summonerCode    float32
6.summonerCode    float32
7.summonerCode    float32
8.summonerCode    float32
9.summonerCode    float32
Length: 1382, dtype: object

In [5]:
# cor = df.corr()
# cor['winningTeam']

Split Data:

Features:

In [6]:
import pandas as pd
import numpy as np
import pymongo
import json
from pandas import read_csv, read_excel
import openpyxl

temp_train=read_excel("data1excel_train.xls")
temp_test=read_excel("data2excel_test.xls")
temp_train.to_csv("data1excel_train.csv")
temp_test.to_csv("data2excel_test.csv")
train_df=read_csv("data1excel_train.csv")
test_df=read_csv("data2excel_test.csv")
train_df=train_df.drop(columns=['Unnamed: 0.1','Unnamed: 0'])
test_df=test_df.drop(columns=['Unnamed: 0.1','Unnamed: 0'])
print(train_df.head())
print(test_df.head())
train_data = train_df
test_data = test_df

Labels:

In [7]:
train_label = train_data.pop('blue_team_won?')
test_label = test_data.pop('blue_team_won?')

Normalize data:

In [8]:
train_stats = train_data.describe().transpose()
train_stats.head()

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,1323.0,994.284180,583.257080,1.0,482.5,993.0,1501.5,2003.0
0.assists,1323.0,5.144369,4.661088,0.0,1.0,4.0,7.0,31.0
0.baronKills,1323.0,0.029478,0.169207,0.0,0.0,0.0,0.0,1.0
0.basicPings,1323.0,11.666667,14.351699,0.0,2.0,7.0,16.0,102.0
0.bountyLevel,1323.0,1.142857,2.277629,0.0,0.0,0.0,1.0,18.0


In [10]:
from sklearn.preprocessing import normalize

norm_test_data = normalize(test_data)
norm_train_data = normalize(train_data)

Build Model:

In [17]:
from tensorflow import keras
from tensorflow.keras import layers

model = tf.keras.models.Sequential([
    layers.Dense(128, activation = 'relu',input_shape = [len(train_data.keys())]),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               176896    
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 193,537
Trainable params: 193,537
Non-trainable params: 0
_________________________________________________________________


Train Model:

If Verbose = 0, use:

In [19]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs):
        if epoch % 100 == 0: print('')
        print('UwU ',end='')

In [20]:
res = model.fit(
    train_data, train_label,
    epochs = 50,
    validation_split = 0.2,
    callbacks = [PrintDot()]
)

Epoch 1/50
34/34 [==============================] - 1s 8ms/step - loss: 1736.1250 - accuracy: 0.7042 - val_loss: 422.9293 - val_accuracy: 0.8566
Epoch 2/50
34/34 [==============================] - 0s 3ms/step - loss: 290.4471 - accuracy: 0.8837 - val_loss: 343.4471 - val_accuracy: 0.8792
Epoch 3/50
34/34 [==============================] - 0s 3ms/step - loss: 306.7695 - accuracy: 0.8705 - val_loss: 531.4819 - val_accuracy: 0.7962
Epoch 4/50
34/34 [==============================] - 0s 3ms/step - loss: 210.2312 - accuracy: 0.8894 - val_loss: 173.7379 - val_accuracy: 0.8830
Epoch 5/50
34/34 [==============================] - 0s 3ms/step - loss: 126.6381 - accuracy: 0.9282 - val_loss: 137.0768 - val_accuracy: 0.9019
Epoch 6/50
34/34 [==============================] - 0s 4ms/step - loss: 63.0983 - accuracy: 0.9442 - val_loss: 129.6760 - val_accuracy: 0.9208
Epoch 7/50
34/34 [==============================] - 0s 3ms/step - loss: 52.7727 - accuracy: 0.9442 - val_loss: 151.6071 - val_accuracy: 

In [21]:
hist = pd.DataFrame(res.history)
hist['epoch'] = res.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
45,1.664594e-18,1.0,94.195541,0.935849,45
46,1.664594e-18,1.0,94.195541,0.935849,46
47,1.664594e-18,1.0,94.195541,0.935849,47
48,1.664594e-18,1.0,94.195541,0.935849,48
49,1.664706e-18,1.0,94.195541,0.935849,49


In [22]:
ev = model.evaluate(test_data,test_label)
print('Model accuracy: ', ev[1])

22/22 [==============================] - 0s 2ms/step - loss: 99.0824 - accuracy: 0.9457
Model accuracy:  0.9456681609153748


Model fitting: